__Generating a Kepler visualisation from a NetCDF file__

In [1]:
from netCDF4 import Dataset
import pandas as pd
import os, glob, pathlib, keplergl

In [2]:
def days_to_timestamps(day_list):
    """
    Convert 'Days since 01-01-1600' into a usable unix timestamp 
    (seconds since 01-01-1970)
    """
    day_diff = 135130       # between 1600 and 1970
    unix_days = [float(d) - day_diff for d in day_list]
    timestamps = [int(round(ud * 86400)) for ud in unix_days]     # convert from days to seconds
    return timestamps


def write_quarters(df, path, key, verbose=True):
    """
    Split Year into Quarters to run more smoothly in the browser.
    If more than ~3 months needs to be visualised at a time then multiple quarters can be
    loaded as layers in Kepler by dragging the CSVs to the web application (kepler.gl).
    Fields will be automatically recognised
    """
    if verbose: print("Writing Quarters...", end="")
    os.chdir(path)
    data_path = pathlib.Path(os.getcwd() + "//" + key[:-4])
    data_path.mkdir(parents=True, exist_ok=True) # make directory  
    os.chdir(data_path)
    
    n = len(df) // 4
    if verbose: print("1...", end="")
    df.iloc[0:n].to_csv("Q1-" + key)
    if verbose: print("2...", end="")
    df.iloc[n:2*n].to_csv("Q2-" + key)
    if verbose: print("3...", end="")
    df.iloc[2*n:3*n].to_csv("Q3-" + key)
    if verbose: print("4...")
    df.iloc[3*n:].to_csv("Q4-" + key)


def process_netCDF(filepath, key=None, verbose=True):
    """
    Process netCDF file into Kepler-readable format. 
    Creates new kepler folder in the file directory (if not already existing) to export to
    Key - filename (before prefixed with Qn-)
    """
    # convert to original-filename.csv if key is not provided 
    key = key if key is not None else pathlib.PurePath(filepath[:-3]).name+".csv"
    assert " " not in key
    key = key if key.endswith(".csv") else key + ".csv"
    df_list = []
    f = Dataset(filepath, "r", format="NETCDF4")
    
    # read in variables
    mt = f.variables['time']
    lat = f.variables['latitude']
    lon = f.variables['longitude']
    height = f.variables['height']
    pressure = f.variables['pressure']
    time = mt[:]
    lat = lat[:]
    lon = lon[:]
    height = height[:]
    pressure = pressure[:]

    data_dict = {"time" : time.tolist(), "lat" : lat.tolist(), "lon" : lon.tolist(), "height" : height.tolist(), "pressure" : pressure.tolist()}
    main_df = pd.DataFrame(data_dict)
    f.close()
    if verbose: print("Adding trajIDs for " + key + "...")
    for i, row in main_df.iterrows():
        temp_dict = {'id': [i] * len(row['time']), 'time': days_to_timestamps(row['time']), 'latitude': row['lat'],
                     'longitude': row['lon'], 'height': row['height'], 'pressure': row['pressure']}
        df = pd.DataFrame(temp_dict)
        df_list.append(df)


    if verbose: print("Concatenating DFs...")
    df_full = pd.concat(df_list, ignore_index=True)

    if verbose: print("Sorting Values...")
    df_full.sort_values(by=['time'], inplace=True)
    
    filedir = os.path.dirname(filepath)
    data_path = pathlib.Path(filedir + "\\kepler\\kepler-data")
    data_path.mkdir(parents=True, exist_ok=True) # make directory                 
    write_quarters(df_full, data_path, key, verbose=verbose)
    
    print("Completed!")             

In [3]:
def process_netCDF_folder(folder):
    """
    Read and process whole folder of .nc files
    """
    os.chdir(folder)
    for file in glob.glob("*.nc"):
        path = os.path.join(folder, file)
        key = pathlib.PurePath(path).name
        process_netCDF(path, key)


### __Processing of NetCDF file__

In [13]:
path = r"D:\FinalNovClim\GroupDev\data\ERA-Interim_1degree_CapeGrim_100m_2016_hourly.nc"     # replace path with own 
process_netCDF(path, key="Cape-Grim-2016")

Adding trajIDs for Cape-Grim-2016.csv...
Concatenating DFs...
Sorting Values...
Writing Quarters...1...2...3...4...
Completed!


### __Processing of NetCDF Folder__

In [80]:
process_netCDF_folder(r"D:\FinalNovClim\GroupDev\data")

## __Kepler.gl__

### __Visualisation within Jupyter__

In [16]:
# Kepler.gl config JSON String (customisable and exportable from within generated Jupyter map)
config = {
  "version": "v1",
  "config": {
    "visState": {
      "filters": [
        {
          "dataId": [
            "Q1-Cape-Grim-2016"
          ],
          "id": "a5ov65eq",
          "name": [
            "time"
          ],
          "type": "timeRange",
          "value": [
            1451606400000,
            1451834847000
          ],
          "enlarged": True,
          "plotType": "histogram",
          "yAxis": None
        }
      ],
      "layers": [
        {
          "id": "u4h1i8r",
          "type": "point",
          "config": {
            "dataId": "Q1-Cape-Grim-2016",
            "label": "Traj-Point",
            "color": [
              18,
              147,
              154
            ],
            "columns": {
              "lat": "latitude",
              "lng": "longitude",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 3.5,
              "fixedRadius": False,
              "opacity": 0.61,
              "outline": False,
              "thickness": 2,
              "strokeColor": None,
              "colorRange": {
                "name": "ColorBrewer RdBu-8",
                "type": "diverging",
                "category": "ColorBrewer",
                "colors": [
                  "#b2182b",
                  "#d6604d",
                  "#f4a582",
                  "#fddbc7",
                  "#d1e5f0",
                  "#92c5de",
                  "#4393c3",
                  "#2166ac"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radiusRange": [
                0,
                50
              ],
              "filled": True
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": {
              "name": "height",
              "type": "integer"
            },
            "colorScale": "quantize",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "sizeField": None,
            "sizeScale": "linear"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "Q1-Cape-Grim-2016": [
              "id",
              "time",
              "height",
              "pressure"
            ]
          },
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "additive",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": -3.068443001905914,
      "dragRotate": True,
      "latitude": -48.52529353459362,
      "longitude": 96.51496428765398,
      "pitch": 23.230709894131998,
      "zoom": 2.2403273800368235,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}

In [7]:
# Read CSV of Quarter
path = r'D:\FinalNovClim\GroupDev\data\kepler\kepler-data\Cape-Grim-2016\Q1-Cape-Grim-2016.csv'
df = pd.read_csv(path, index_col=0)

# generate map from CSV and Config
w1 = keplergl.KeplerGl(height=500, data={"Q1-Cape-Grim-2016": df}, config=config)
# w1.add_data(df2, 'Q2-Cape-Grim-2016')          # additional CSVs can be added if required

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [22]:
w1

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['Q1-Cape-Grim-2016'], 'id': …

### __Export to Portable HTML file__

In [15]:
w1.save_to_html(file_name="Cape_Grim_1.html", data={"Q1-Cape-Grim-2016": df}, config=config)

Map saved to Cape_Grim_1.html!
